## Simple is better than complex

### Reverse a List

In [1]:
my_list = [1, 2, 3, 4, 5, 6]

In [2]:
reversed_list = []

for x in range(len(my_list)):
    reversed_list.append(my_list[len(my_list)-x-1])
    
print(reversed_list)

[6, 5, 4, 3, 2, 1]


In [5]:
my_list[::-1]

[6, 5, 4, 3, 2, 1]

### Parsing known format

In [ ]:
def parse_file(self):
        """The following lines parse through an sxst"""
        level = 0
        cyme_class = ''
        cyme_subclass = ''
        object_field_values = {}
        subobject_field_values = {}
        phase = ''

        # Variable for forwards and reverse settings in the
        # Regulator class
        settings = None

        for line in self.cyme_file:

            tag = line.split('>')[0].strip()[1:]

            if tag in dictof_input_cyme_objectpropertties:
                level += 1
                if level == 1:
                    cyme_class = tag
                    if cyme_class not in self.cyme_dict:
                        self.cyme_dict[cyme_class] = {}
                    object_field_values.update(dict.fromkeys(
                        dictof_output_cyme_objectpropertties[
                            cyme_class], ''))
                if level == 2:

                    cyme_subclass = tag
                    object_name = object_field_values[
                        dictof_input_cyme_objectpropertties[
                            cyme_class][0]]
                    if object_name not in self.cyme_dict[cyme_class]:
                        self.cyme_dict[cyme_class][object_name] = {}
                    if cyme_subclass not in \
                            self.cyme_dict[cyme_class][object_name]:
                        self.cyme_dict[
                            cyme_class][object_name][cyme_subclass] = {}
                    subobject_field_values.update(dict.fromkeys(
                        dictof_output_cyme_objectpropertties[
                            cyme_subclass], ''))
            elif level == 1 and tag in \
                    dictof_input_cyme_objectpropertties[cyme_class]:

                val = self.get_tag_value(line)[1]

                # In cyme, PositionOfConductorX is a
                # subclass of OverheadSpacingOfConductorDB class.

                if cyme_class == 'OverheadSpacingOfConductorDB' \
                        and tag.startswith('PositionOfConductor'):
                    position_of_conductor = tag
                elif cyme_class == 'OverheadSpacingOfConductorDB' \
                        and tag in ['X', 'Y']:
                    object_field_values[position_of_conductor + '_' + tag] = \
                        val
                else:
                    object_field_values[tag] = val
            elif level == 2 and tag in \
                    dictof_input_cyme_objectpropertties[cyme_subclass]:

                val = self.get_tag_value(line)[1]

                # In cyme multiple load models are given
                # for SpotLoad and DistributedLoad sub-objects. Load values are
                # only read for LodeModelID = 1.
                if cyme_subclass in ['SpotLoad', 'DistributedLoad'] \
                        and tag == 'LoadModelID':

                    if val == '1':
                        flag_read_load_value = True
                        subobject_field_values[tag] = val
                    else:
                        flag_read_load_value = False

                # LoadValueType property is only recorded for LoadModelID = 1.
                elif cyme_subclass in ['SpotLoad', 'DistributedLoad'] \
                        and tag == 'LoadValueType' and flag_read_load_value:
                    subobject_field_values[tag] = val
                # SpotLoad and DistributedLoad KVA, KW, KVAR, and PF
                # can be specified by phase or three phase.
                # These properties are only recorded for LoadModelID = 1.
                elif cyme_subclass in ['SpotLoad', 'DistributedLoad'] \
                        and tag == 'Phase' and flag_read_load_value:
                    phase = val
                elif cyme_subclass in ['SpotLoad', 'DistributedLoad'] \
                        and (tag in ['KVA', 'KW', 'KVAR', 'PF']) \
                        and len(phase) == 1 and flag_read_load_value:
                    subobject_field_values[tag + phase] = val
                elif cyme_subclass in ['SpotLoad', 'DistributedLoad'] \
                        and (tag in ['KVA', 'KW', 'KVAR', 'PF']) \
                        and len(phase) == 3 and flag_read_load_value:
                    if tag == 'PF':
                        subobject_field_values[tag + 'A'] = val
                        subobject_field_values[tag + 'B'] = val
                        subobject_field_values[tag + 'C'] = val
                    else:
                        subobject_field_values[tag + 'A'] = str(float(val)/3)
                        subobject_field_values[tag + 'B'] = str(float(val)/3)
                        subobject_field_values[tag + 'C'] = str(float(val)/3)

                # Load value properties for LoadModelIDs
                # different from 1 are ignored.

                elif cyme_subclass in ['SpotLoad', 'DistributedLoad'] \
                    and (tag in ['LoadValueType', 'Phase',
                                 'KVA', 'KW', 'KVAR', 'PF']) and not \
                        flag_read_load_value:
                    continue

                # In cyme, ForwardSettings and
                # ReverseSettings are a subclass of Regulator class.

                # ========================================================
                elif cyme_subclass == 'Regulator' and tag == 'ForwardSettings':
                    settings = 'Forward'
                elif cyme_subclass == 'Regulator' and tag == 'ReverseSettings':
                    settings = 'Reverse'
                elif cyme_subclass == \
                    'Regulator' and tag in ['RsetA', 'RsetB', 'RsetC',
                                            'XsetA', 'XsetB', 'XsetC',
                                            'SetVoltageA', 'SetVoltageB',
                                            'SetVoltageC', 'BandwidthA',
                                            'BandwidthB', 'BandwidthC']:
                    subobject_field_values[settings + tag] = val
                # ========================================================
                else:
                    subobject_field_values[tag] = val
            elif tag.startswith('/') and tag[1:] in \
                    dictof_input_cyme_objectpropertties:
                if level == 1:
                    object_name = object_field_values[
                        dictof_input_cyme_objectpropertties[
                            cyme_class][0]]
                    if object_name in self.cyme_dict[cyme_class]:
                        self.cyme_dict[cyme_class][object_name].update(
                            object_field_values)
                    else:
                        self.cyme_dict[cyme_class][object_name] = \
                            object_field_values
                    object_field_values = {}
                    level += -1
                if level == 2:
                    object_name = \
                        object_field_values[
                            dictof_input_cyme_objectpropertties[
                                cyme_class][0]]
                    subobject_name = \
                        subobject_field_values[
                            dictof_input_cyme_objectpropertties[
                                cyme_subclass][0]]
                    self.cyme_dict[cyme_class][object_name][
                        cyme_subclass][subobject_name] = subobject_field_values
                    subobject_field_values = {}
                    level += -1

        return self.cyme_dict

In [ ]:
import xmltodict

cyme_dict = xmltodict.parse(cyme_file)